In [1]:
from skfeature.function.similarity_based import fisher_score
from scipy.linalg import norm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2 as cv
import glob
import os
import glob
import cv2 as cv
import pandas as pd
import pickle
import os
from random import choices
import numpy as np
from sklearn.utils import shuffle
from sklearn.neural_network import MLPClassifier
from scipy.stats import mode
from utilities import test
from utilities import move
from image_process import preprocess
from image_process import appply_filter
from sklearn.metrics import ConfusionMatrixDisplay

In [2]:
def preprocess(img,size=(32,32)):
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    resized=cv.resize(gray,size)
    clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    normalized = clahe.apply(resized)
    return normalized
    
def appply_filter(img,kernels,lables):
    df=pd.DataFrame()
    df['Original Image']=img.reshape(-1)
    raw_features=[]
    for kernel,label in zip(kernels,lables):
        fimg = cv.filter2D(img,cv.CV_8UC3,kernel)
        raw_features.append(fimg)
        df[label]=fimg.reshape(-1)
    return df,raw_features 

def form_data(frame,kernels=None,lables=None,size=(32,32)):
    if kernels==None or lables==None:
        kernels,lables=generatte_filters()
    X=[]
    Y=[]
    for i in range(frame.shape[0]):
        path=os.path.relpath(frame.iloc[i]['file'])
        im=cv.imread(path)
        try:
            len(im)
        except:
            print(path)
            continue
        cls=[int(frame.iloc[i]['class'])]
        proc_im=preprocess(im,size)
        data_f,raw_features=appply_filter(proc_im,kernels,lables)
        feat_array=np.array(data_f.to_numpy().T)
        X.append(feat_array)
        Y.append(cls*data_f.shape[1])
    X=np.array(X)
    Y=np.array(Y)
    X=X.reshape(-1,X.shape[-1])
    Y=Y.reshape(-1)
    return X,Y

In [3]:
def train(model,kernels,lables):
    Xfull,Yfull=form_data(read_images(),kernels,kernels)
    model.fit(Xfull,Yfull)
    #end=time.time()-start

In [4]:
def calculate_score(X,Y,norm_type=np.inf):
    X=np.array(X)
    Y=np.array(Y)
    i=0
    scores=[]
    for k in range(0,X.shape[1]):
        i=i+1
        m1k=X[:,k][Y==0].mean(0)
        m2k=X[:,k][Y==1].mean(0)
        sig1k=X[:,k][Y==0].std(0)
        sig2k=X[:,k][Y==1].std(0)
        j=(norm(m1k-m2k,norm_type)**2)/(norm(sig1k**2,norm_type)+norm(sig2k**2,norm_type))
        scores.append(j)
    return scores

In [5]:
def calculate_score(X,Y,norm_type=np.inf):
    X=np.array(X)
    Y=np.array(Y)
    i=0
    scores=[]
    for k in range(0,X.shape[1]):
        i=i+1
        m1k=X[:,k][Y==0].mean(0)
        m2k=X[:,k][Y==1].mean(0)
        sig1k=X[:,k][Y==0].std(0)
        sig2k=X[:,k][Y==1].std(0)
        j=(norm(m1k-m2k,norm_type)**2)/(norm(sig1k**2,norm_type)+norm(sig2k**2,norm_type))
        scores.append(j)
    return scores

def select_filters(X,Y,all_kernels,all_lables,number_of_filters =30):
    scores=calculate_score(X,Y);
    ScoreBoard=pd.DataFrame(scores,[i for i in range(0,len(all_kernels))])
    ScoreBoard=ScoreBoard.dropna()
    ScoreBoard=ScoreBoard.sort_values(by=0)
    selected_kernels=[all_kernels[i] for i in ScoreBoard.iloc[-number_of_filters:].index]
    selsected_lables=[all_lables[i] for i in ScoreBoard.iloc[-number_of_filters:].index]
    return selected_kernels,selsected_lables